<a href="https://colab.research.google.com/github/dini-5002/Grammar_check/blob/main/subverb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Sample dataset
data = [
    ("He go to the market.", 1),
    ("She goes to the market.", 0),
    ("They is playing soccer.", 1),
    ("They are playing soccer.", 0)
]

df = pd.DataFrame(data, columns=['sentence', 'label'])

# Splitting data
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['sentence'])

X_train = tokenizer.texts_to_sequences(train_data['sentence'])
X_test = tokenizer.texts_to_sequences(test_data['sentence'])

# Padding sequences to the same length
max_len = 10
X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
X_test = pad_sequences(X_test, padding='post', maxlen=max_len)

y_train = train_data['label'].values
y_test = test_data['label'].values

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Model definition
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_len))
model.add(LSTM(units=64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 64)            704       
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 33793 (132.00 KB)
Trainable params: 33793 (132.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)



Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.6935 - accuracy: 0.5000 - val_loss: 0.6993 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 82ms/step - loss: 0.6920 - accuracy: 0.5000 - val_loss: 0.6999 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 72ms/step - loss: 0.6904 - accuracy: 0.5000 - val_loss: 0.7008 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 83ms/step - loss: 0.6886 - accuracy: 1.0000 - val_loss: 0.7022 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 69ms/step - loss: 0.6867 - accuracy: 1.0000 - val_loss: 0.7043 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 84ms/step - loss: 0.6844 - accuracy: 1.0000 - val_loss: 0.7071 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 72ms/step - loss: 0.6817 - accuracy: 1.0000 - val_loss: 0.7109 - val_accuracy: 0.0000e+00
E

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


1/1 [==============================] - 0s 30ms/step - loss: 0.6823 - accuracy: 1.0000
Test Accuracy: 100.00%


In [ ]:
def predict_error(sentence):
    pos_tags = pos_tagging(sentence)
    encoded_tags = pos_encoder.transform([tag for word, tag in pos_tags])
    padded_tags = pad_sequences([encoded_tags], maxlen=max_len, padding='post')
    prediction = model.predict(padded_tags)
    return "Error" if prediction.argmax() == 1 else "No Error"

# Example usage
sentence = "He go to the market."
print(predict_error(sentence))  # Output should be "Error"


1/1 [==============================] - 1s 807ms/step
Error
